In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, gc, time, warnings, pickle, psutil, random
from google.colab import drive
from IPython.core.interactiveshell import InteractiveShell
import plotly.offline
import plotly.express as px
import gc
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100
InteractiveShell.ast_node_interactivity = "all"

from  datetime import datetime, timedelta
import lightgbm as lgb
from lightgbm.callback import early_stopping

warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
os.chdir('/content/gdrive/MyDrive/msba6421_predictive/Amy_group_project/m5-forecasting-accuracy')

In [ ]:
gc.collect()

18

# Training By store_id - cat_id

In [ ]:
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
DATA = '1202_data.pkl'
dir_ = '/content/gdrive/MyDrive/msba6421_predictive/Amy_group_project/m5-forecasting-accuracy'
model_dir = 'model/'
log_dir = 'log/'
submission_dir = 'submission/'

STORES = [
    'CA_1', 'CA_2',
    'CA_3',
    'CA_4'
    'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3'
    ]
DEPTS = ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']

TARGET = 'sold'

STEPS = list(np.arange(4,30,4))   #[7, 14, 21, 28]
HORIZON = 28
TRAIN_START = 1
TRAIN_END = 1941 - HORIZON
FIRST_VAL_DAY = TRAIN_END + 1
LAST_VAL_DAY = 1941
FIRST_PRED_DAY = 1941 + 1
VAL_START = 1942-HORIZON
VAL_END = 1941
VAL_DAYS, TEST_DAYS = STEPS[0], STEPS[0]

BASE = pd.read_pickle(DATA)
remove_feature = ['id',
                  'state_id',
                  'store_id',
                  'item_id',
                  'dept_id',
                  'cat_id',
                  'date','wm_yr_wk','d','sold']

lags_col = list(BASE.columns[(BASE.columns.str.contains('lag')) | (BASE.columns.str.contains('rm_')) | (BASE.columns.str.contains('std_')) | (BASE.columns.str.contains('max_')) | (BASE.columns.str.contains('ema_'))])
cat_var = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
cat_var = list(set(cat_var) - set(remove_feature))

In [ ]:
# Make training data

def prepare_data(store, state):
    grid_df = BASE[(BASE['store_id'] == store) & (BASE['dept_id'] == state)]

    grid_df = reduce_mem_usage(grid_df)

    return grid_df

In [ ]:
# Model params
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1000,
                    'boost_from_average': False,
                    'verbose': -1,
                    'seed' : 1995
                }

## Start training + prediction

In [ ]:
predictions = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_temp_3_1.pkl')

for store in STORES:
    for state in DEPTS:
        for step in STEPS:
          print(store,state, 'start')
          grid_df = prepare_data(store, state)
          grid_df[lags_col] = grid_df.groupby(['id'], observed=False)[lags_col].shift(step)
          model_var = grid_df.columns[~grid_df.columns.isin(remove_feature)]
          # ix_to_drop = grid_df[(grid_df['d'] <= 1941) & grid_df.isna().any(axis=1)].index
          # grid_df.drop(index=ix_to_drop, inplace=True)

          pred_start = FIRST_PRED_DAY + step - VAL_DAYS
          pred_end = FIRST_PRED_DAY + step - 1


          tr_mask = (grid_df['d'] >= TRAIN_START) & (grid_df['d'] <= TRAIN_END)
          vl_mask = (grid_df['d'] >= VAL_START) & (grid_df['d'] <= VAL_END)
          pr_mask = (grid_df['d'] >= pred_start) & (grid_df['d'] <= pred_end)

          trainX = grid_df[tr_mask][model_var]
          trainY = grid_df[tr_mask][TARGET]
          valX = grid_df[vl_mask][model_var]
          valY = grid_df[vl_mask][TARGET]
          testX = grid_df[pr_mask][model_var]
          print(f'Train shape: {trainX.shape}. Val shape: {valX.shape}. Test shape: {testX.shape}')

          # Train
          lgbm = lgb.LGBMRegressor(**lgb_params)
          callbacks = [early_stopping(stopping_rounds=50, first_metric_only=False)]

          lgbm.fit(trainX, trainY,
                  eval_set=[(valX, valY)],
                  eval_metric='rmse',
                  callbacks=callbacks)

          # Predict
          yhat = lgbm.predict(testX, num_iteration=lgbm.best_iteration_)
          preds = grid_df[(grid_df['d'] >= pred_start) & (grid_df['d'] <= pred_end)][['id', 'd']]
          preds['sales'] = yhat
          predictions = pd.concat([predictions, preds], axis=0)
          predictions.to_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_temp_3_1.pkl')
          del grid_df, trainX, trainY, valX, valY, testX, lgbm, tr_mask, vl_mask, pr_mask ; gc.collect

CA_3 HOBBIES_1 start
Train shape: (651279, 74). Val shape: (11648, 74). Test shape: (1664, 74)
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[155]	valid_0's rmse: 2.44158


LGBMRegressor(boost_from_average=False, feature_fraction=0.5,
              learning_rate=0.015, max_bin=100, metric='rmse',
              min_data_in_leaf=255, n_estimators=1000, num_leaves=255,
              objective='tweedie', seed=1995, subsample=0.5, subsample_freq=1,
              tweedie_variance_power=1.1, verbose=-1)

<function gc.collect(generation=2)>

CA_3 HOBBIES_1 start
Train shape: (651279, 74). Val shape: (11648, 74). Test shape: (1664, 74)
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[155]	valid_0's rmse: 2.43193


LGBMRegressor(boost_from_average=False, feature_fraction=0.5,
              learning_rate=0.015, max_bin=100, metric='rmse',
              min_data_in_leaf=255, n_estimators=1000, num_leaves=255,
              objective='tweedie', seed=1995, subsample=0.5, subsample_freq=1,
              tweedie_variance_power=1.1, verbose=-1)

...

LGBMRegressor(boost_from_average=False, feature_fraction=0.5,
              learning_rate=0.015, max_bin=100, metric='rmse',
              min_data_in_leaf=255, n_estimators=1000, num_leaves=255,
              objective='tweedie', seed=1995, subsample=0.5, subsample_freq=1,
              tweedie_variance_power=1.1, verbose=-1)

<function gc.collect(generation=2)>

CA_4 FOODS_3 start
Train shape: (1237769, 74). Val shape: (23044, 74). Test shape: (3292, 74)
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[463]	valid_0's rmse: 1.77557


LGBMRegressor(boost_from_average=False, feature_fraction=0.5,
              learning_rate=0.015, max_bin=100, metric='rmse',
              min_data_in_leaf=255, n_estimators=1000, num_leaves=255,
              objective='tweedie', seed=1995, subsample=0.5, subsample_freq=1,
              tweedie_variance_power=1.1, verbose=-1)

<function gc.collect(generation=2)>

In [ ]:
predictions.to_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_temp_3_1.pkl')

# Submission

In [ ]:
df_dept_temp1 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_temp_3_1.pkl')
df_dept_temp2 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_3_2.pkl')
# df_dept_miss1 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_temp_2_1_missing.pkl')
# df_dept_miss2 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_temp_2_2_missing.pkl')

df_cat_temp1 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_cat_preds_temp_3_1.pkl')
df_cat_temp2 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_cat_preds_temp_3_2.pkl')
# df_cat_miss1 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_cat_preds_temp_2_1_missing.pkl')
# df_cat_miss2 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_cat_preds_temp_2_2_missing.pkl')

df_dept = pd.concat([df_dept_temp1, df_dept_temp2], axis=0)
# df_dept = pd.concat([df_dept, df_dept_temp2], axis=0)
# df_dept = pd.concat([df_dept, df_dept_miss2], axis=0)

df_cat = pd.concat([df_cat_temp1, df_cat_temp2], axis=0)
# df_cat = pd.concat([df_cat, df_cat_temp2], axis=0)
# df_cat = pd.concat([df_cat, df_cat_miss2], axis=0)

df_dept = df_dept.groupby(['id','d']).head(1).sort_values(by = ['id','d'])
df_cat = df_cat.groupby(['id','d']).head(1).sort_values(by = ['id','d'])

In [ ]:
df_dept.to_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_3.pkl')
df_cat.to_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_cat_preds_3.pkl')

In [ ]:
pred1 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_depts_preds_3.pkl')
pred2 = pd.read_pickle(f'{submission_dir}before_ensemble/me_with_steps_store_cat_preds_3.pkl')


pred1.set_index(['id', 'd'], inplace=True)
pred2.set_index(['id', 'd'], inplace=True)

df_avg = (pred1 + pred2) / 2
# df_avg = pred1
df_avg.reset_index(inplace=True)

submission = pd.read_csv('sample_submission.csv')
df_avg = df_avg.pivot(index='id', columns='d', values='sales').reset_index()
df_avg.columns = submission.columns
df_avg = submission[['id']].merge(df_avg, on='id', how='left').fillna(1)
print(df_avg)
submission_file = f'{submission_dir}me_with_step_submission_3.csv'
df_avg.to_csv(submission_file, index=False)

                                  id        F1        F2        F3        F4  \
0      HOBBIES_1_001_CA_1_validation  1.000000  1.000000  1.000000  1.000000   
1      HOBBIES_1_002_CA_1_validation  1.000000  1.000000  1.000000  1.000000   
2      HOBBIES_1_003_CA_1_validation  1.000000  1.000000  1.000000  1.000000   
3      HOBBIES_1_004_CA_1_validation  1.000000  1.000000  1.000000  1.000000   
4      HOBBIES_1_005_CA_1_validation  1.000000  1.000000  1.000000  1.000000   
...                              ...       ...       ...       ...       ...   
60975    FOODS_3_823_WI_3_evaluation  0.495422  0.439658  0.443123  0.436633   
60976    FOODS_3_824_WI_3_evaluation  0.258992  0.242010  0.259664  0.255498   
60977    FOODS_3_825_WI_3_evaluation  0.618491  0.598668  0.556558  0.585791   
60978    FOODS_3_826_WI_3_evaluation  1.076119  0.990626  0.935592  0.885062   
60979    FOODS_3_827_WI_3_evaluation  1.269679  1.186004  1.225097  1.243173   

             F5        F6        F7    